In [4]:
from openai import OpenAI
client = OpenAI(api_key='sk-proj-0SPGwDGsnFum7WS2wMGpT3BlbkFJwdLZHeFJcEa2gSdCuIFV')

def get_embedding(text, model="text-embedding-3-small"):
   text = text.replace("\n", " ")
   return client.embeddings.create(input = [text], model=model).data[0].embedding

In [1]:
import json
file_path='data/law_data/new_법령이름.json'
with open(file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

In [2]:
import json
file_path='data/law_data/new_질문&법령.json'
with open(file_path, 'r', encoding='utf-8') as file:
    data1 = json.load(file)

In [18]:
list1=[]
for i in range(len(data)):
    list1.append(get_embedding(data[i]['법내용']))

In [101]:
from tqdm import tqdm
list2=[]
for i in tqdm(range(len(data1))):
    list2.append(get_embedding(data1[i]['질문']))

100%|██████████| 5802/5802 [29:03<00:00,  3.33it/s] 


In [19]:
import pickle
import numpy as np
embeddings=np.array(list1)
with open('embeddings.pkl', 'wb') as f:
    pickle.dump(embeddings, f)

In [102]:
import pickle
import numpy as np
embeddings=np.array(list2)
with open('q_embeddings.pkl', 'wb') as f:
    pickle.dump(embeddings, f)

In [6]:
import pickle
import faiss

# pkl 파일에서 임베딩 로드
with open('embeddings.pkl', 'rb') as f:
    embeddings = pickle.load(f)

# Faiss 인덱스 생성
dimension = embeddings.shape[1]# 임베딩의 차원
index = faiss.IndexFlatL2(dimension)  # L2 거리 사용

# 임베딩을 Faiss 인덱스에 추가
index.add(embeddings)

#predict 
def predict(question,k,index):
    distances, indices = index.search(question, k)
    return indices[0]

def get_preds(predict_number, data):
    preds=[]
    for i in predict_number:
        preds.append(data[i]['법이름'])
    return preds

def test(index,k,data,data1):
    with open('q_embeddings.pkl', 'rb') as f:
        q_list = pickle.load(f)
    total=0
    for q_i, q in enumerate(tqdm(q_list)):
        predict_number=predict(q.reshape(1,-1),k,index)
        preds=get_preds(predict_number, data)
        total+=recall(data1[q_i]['법령'],preds)
    return total/len(q_list)

In [65]:
#recall k 
def recall(answers, preds):
    t=0
    for ans in answers:
        if ans in preds:
            t+=1
    
    return t/len(answers)

In [105]:
def test(index,k,data,data1):
    with open('q_embeddings.pkl', 'rb') as f:
        q_list = pickle.load(f)
    total=0
    for q_i, q in enumerate(tqdm(q_list)):
        predict_number=predict(q.reshape(1,-1),k,index)
        preds=get_preds(predict_number, data)
        total+=recall(data1[q_i]['법령'],preds)
    return total/len(q_list)

In [ ]:
len(bm25_result

In [112]:
test(index,100,data,data1)

100%|██████████| 5802/5802 [00:03<00:00, 1681.01it/s]


0.6622247335471223

In [ ]:
client = OpenAI(
    api_key = "sk-proj-0SPGwDGsnFum7WS2wMGpT3BlbkFJwdLZHeFJcEa2gSdCuIFV"
)



In [137]:
for i in tqdm(range(len(data))):   
    text_info = data[i]['법내용']
    prompt=f'''
    #TEXT{text_info}
    # TASK
    I will provide you with a document. Your task is to generate five questions that are related to the content of the document. 
    These questions should help in understanding the main ideas, themes, and important details within the document. 
    Ensure the questions are diverse and cover different aspects of the document, such as the main argument, supporting details, implications, and any interesting points raised. 
    It must be written in Korean. 
    Here is the document:'''

    message = [
        {"role": "user", "content": prompt},
    ]

    request_data = {
        "messages": message,
        "max_tokens": 250,
        "temperature": 0,
        "top_p": 1,
        "n": 1,
        "stream": False,
    }


    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        **request_data,
    )

    # data[i]['관련질의']=response.choices[0].message.content
    a=response.choices[0].message.content
    break
a

100%|██████████| 1464/1464 [41:59<00:00,  1.72s/it]
